<a href="https://colab.research.google.com/github/Barros313/Big-Data-1---Senac-TADS028/blob/main/BigData1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação do Banco de Dados

Importação de bibliotecas externas e criação de função para fazer o banco de dados em SQL a partir de banco de dados em CSV

In [5]:
import requests
import sqlite3
import pandas as pd

def create_infracoes_database():
  url = 'http://dados.recife.pe.gov.br/api/3/action/datastore_search?resource_id=c269789d-da47-4dde-8ce7-42fba10fe8e2&limit=470000'

  response = requests.get(url)
  data = response.json()

  if 'result' in data and 'records' in data['result']:
    infracoes = data['result']['records']
  else:
    print("Erro: não foi possível obter dados da API")
    return

  conn = sqlite3.connect('infracoes.db')
  c = conn.cursor()

  c.execute('''
    CREATE TABLE IF NOT EXISTS infracoes (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      datainfracao DATE,
      horainfracao TIMESTAMP,
      dataimplantacao DATETIME,
      agenteequipamento TEXT,
      infracao INT,
      descricaoinfracao TEXT,
      amparolegal TEXT,
      localcometimento TEXT
    )
  ''')

  for infracao in infracoes:
    c.execute('''
      INSERT INTO infracoes (
        datainfracao,
        horainfracao,
        dataimplantacao,
        agenteequipamento,
        infracao,
        descricaoinfracao,
        amparolegal,
        localcometimento)
      VALUES (?, ?, ?, ?, ?, ?, ?, ?);
    ''', (
        infracao.get("datainfracao", ""),
        infracao.get("horainfracao", ""),
        infracao.get("dataimplantacao", ""),
        infracao.get("agenteequipamento", ""),
        infracao.get("infracao", 0),
        infracao.get("descricaoinfracao", ""),
        infracao.get("amparolegal", ""),
        infracao.get("localcometimento", "")
        ))

  conn.commit()
  conn.close()

create_infracoes_database()



# Consulta geral

Consultando tabela única (infrações) do banco de dados

In [6]:
connection = sqlite3.connect('infracoes.db')

pd.read_sql("SELECT * FROM infracoes;", connection, index_col=["id"])

,datainfracao,horainfracao,dataimplantacao,agenteequipamento,infracao,descricaoinfracao,amparolegal,localcometimento
id,,,,,,,,
1,2023-01-01T00:00:00,00:01:00,2023-01-06T00:00:00,Código 8 - AUTOS NO TALÃO ELETRÔNICO,7234,Quando o veículo estiver em movimento deixar d...,"Art. 250, Inc. I, alínea a","RUA RIBEIRO DE BRITO, SOB O SEMAFORO N. 155"
2,2023-01-01T00:00:00,00:27:00,2023-01-06T00:00:00,Código 8 - AUTOS NO TALÃO ELETRÔNICO,5835,Desobedecer às ordens emanadas da autoridade c...,Art. 195,"RUA DOS NAVEGANTES, CRUZAMENTO COM A RUA PADRE..."
3,2023-01-01T00:00:00,00:27:00,2023-01-06T00:00:00,Código 8 - AUTOS NO TALÃO ELETRÔNICO,5215,Dirigir ameaçando os pedestres que estejam atr...,Art. 170,"RUA DOS NAVEGANTES, CRUZAMENTO COM A RUA PADRE..."
4,2023-01-01T00:00:00,00:52:00,2023-01-06T00:00:00,Código 8 - AUTOS NO TALÃO ELETRÔNICO,5550,Estacionar o veículo em locais e horários proi...,"Art. 181, Inc. XVIII","AVENIDA ENGENHEIRO DOMINGOS FERREIRA, EM FRENT..."
5,2023-01-01T00:00:00,00:52:00,2023-01-06T00:00:00,Código 8 - AUTOS NO TALÃO ELETRÔNICO,5550,Estacionar o veículo em locais e horários proi...,"Art. 181, Inc. XVIII","AVENIDA ENGENHEIRO DOMINGOS FERREIRA, EM FRENT..."
...,...,...,...,...,...,...,...,...
459709,2023-11-30T00:00:00,21:48:51,2023-12-06T00:00:00,Código 5 - FOTO SENSOR,5746,Transitar em locais e horários não permitidos ...,"Art. 187, Inc. I","AV. CDE. DA BOA VISTA, SEMAFORO 486, Sentido:C..."
459710,2023-11-30T00:00:00,21:51:48,2023-12-06T00:00:00,Código 5 - FOTO SENSOR,5746,Transitar em locais e horários não permitidos ...,"Art. 187, Inc. I","AV. CDE. DA BOA VISTA, SEMAFORO 486, Sentido:C..."
459711,2023-11-30T00:00:00,21:59:00,2023-12-11T00:00:00,Código 7 - ZONA AZUL TALÃO ELETRÔNICO,5541,Estacionar o veículo em desacordo com as condi...,"Art. 181, Inc. XVII","PRACA DO DERBY, LADO OPOSTO AO N. 149"


# Funções de query SQL

Função de select geral para utilizar no pandas

In [10]:
infra = pd.read_sql_query("SELECT * FROM infracoes;", connection)

# Características da Tabela

Comando para descrever os dados da tabela

|Output|Descrição|
|---|---|
|count|Quantidade de Registros|
|mean|Média|
|std|Desvio Padrão|
|min|Valor mínimo|
|max|Valor máximo|
|n%|Valor percentual em relação ao máximo|

In [11]:
infra.describe()

,id,infracao
count,459713.000000,459713.000000
mean,229857.000000,6607.109107
std,132707.856487,913.657366
min,1.000000,5010.000000
25%,114929.000000,5550.000000
50%,229857.000000,7234.000000
75%,344785.000000,7455.000000
max,459713.000000,7765.000000


### Informações

Informa se valores há valores nulos, o tipo da célula e o nome das colunas


In [8]:
infra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459713 entries, 0 to 459712
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   id                 459713 non-null  int64 
 1   datainfracao       459713 non-null  object
 2   horainfracao       459713 non-null  object
 3   dataimplantacao    459713 non-null  object
 4   agenteequipamento  459713 non-null  object
 5   infracao           459713 non-null  int64 
 6   descricaoinfracao  459713 non-null  object
 7   amparolegal        459713 non-null  object
 8   localcometimento   459713 non-null  object
dtypes: int64(2), object(7)
memory usage: 31.6+ MB


In [14]:
primeiro_registro = pd.to_datetime(infra['datainfracao']).dt.date.min()
print("Primeiro registro: ", primeiro_registro)

ultimo_registro = pd.to_datetime(infra['datainfracao']).dt.date.max()
print("Último registro: ", ultimo_registro)

Primeiro registro:  2023-01-01
Último registro:  2023-11-30


In [15]:
print(infra.isna().value_counts())


id     datainfracao  horainfracao  dataimplantacao  agenteequipamento  infracao  descricaoinfracao  amparolegal  localcometimento
False  False         False         False            False              False     False              False        False               459713
Name: count, dtype: int64


In [17]:
print(infra.isnull().value_counts())

id     datainfracao  horainfracao  dataimplantacao  agenteequipamento  infracao  descricaoinfracao  amparolegal  localcometimento
False  False         False         False            False              False     False              False        False               459713
Name: count, dtype: int64
